# [PyTorch Tutorial](https://pytorch.org/tutorials/beginner/basics/intro.html)

In [12]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

## Load Data

In [13]:
training_data = datasets.FashionMNIST(
    root="../data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="../data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

## Build Model

References:

* [How to build a Model](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)
* [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)

### Model Class

In [14]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    # See https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.forward
    # for when to call it and when not
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# NOTE: It seems moving model and inputs to "cuda" doesn't improve the overall performance 
# of this simple implemetation at all, comparing to "cpu"!
model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [15]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"{logits = }\n{logits.sum() = }\n{pred_probab = }\n{pred_probab.sum() = }\n{y_pred =}")

logits = tensor([[-0.0928, -0.0721,  0.0883, -0.0428, -0.0174, -0.0752,  0.0089, -0.0919,
          0.0171,  0.0340]], device='cuda:0', grad_fn=<AddmmBackward0>)
logits.sum() = tensor(-0.2439, device='cuda:0', grad_fn=<SumBackward0>)
pred_probab = tensor([[0.0932, 0.0952, 0.1117, 0.0980, 0.1005, 0.0949, 0.1032, 0.0933, 0.1041,
         0.1058]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
pred_probab.sum() = tensor(1.0000, device='cuda:0', grad_fn=<SumBackward0>)
y_pred =tensor([2], device='cuda:0')


### Model Layers

In [16]:
input_image = torch.rand(3,28,28)
print(input_image.size())
# print(input_image)

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())
# print(flat_image)

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())
# print(hidden1)

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

torch.Size([3, 28, 28])
torch.Size([3, 784])
torch.Size([3, 20])
Before ReLU: tensor([[ 0.1093,  0.1109, -0.1638,  0.3122, -0.0237, -0.4138, -0.0963, -0.5451,
          0.3469,  0.2707, -0.1301, -0.1254,  0.0143, -0.5955,  0.0356,  0.2761,
          0.1272,  0.2938, -0.1693,  0.0664],
        [-0.2553,  0.2270, -0.1184,  0.3061,  0.0158,  0.1994, -0.4287, -0.6827,
          0.1763,  0.6024, -0.5433, -0.4022,  0.0729, -0.1965,  0.0909,  0.4221,
          0.0829,  0.4132, -0.5167, -0.1717],
        [-0.0662,  0.0429, -0.2325, -0.2691, -0.1363,  0.0087, -0.1470, -0.5717,
          0.2982,  0.7517,  0.0679, -0.0732, -0.0555, -0.2251,  0.1919,  0.4112,
          0.0835,  0.2957, -0.2653, -0.0636]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1093, 0.1109, 0.0000, 0.3122, 0.0000, 0.0000, 0.0000, 0.0000, 0.3469,
         0.2707, 0.0000, 0.0000, 0.0143, 0.0000, 0.0356, 0.2761, 0.1272, 0.2938,
         0.0000, 0.0664],
        [0.0000, 0.2270, 0.0000, 0.3061, 0.0158, 0.1994, 0.0000, 0.00

In [17]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(f"{logits = }\n{pred_probab = }")


logits = tensor([[-0.0024,  0.1121,  0.2538, -0.1514,  0.2429,  0.0407, -0.1362, -0.3547,
          0.3516,  0.2262],
        [-0.0178,  0.1357,  0.2081, -0.0730,  0.2568,  0.0254, -0.1753, -0.3227,
          0.3872,  0.1941],
        [-0.0193,  0.0783,  0.1755, -0.0499,  0.2726,  0.0820, -0.2154, -0.2349,
          0.2956,  0.0972]], grad_fn=<AddmmBackward0>)
pred_probab = tensor([[0.0921, 0.1033, 0.1190, 0.0794, 0.1177, 0.0962, 0.0806, 0.0648, 0.1312,
         0.1158],
        [0.0905, 0.1055, 0.1134, 0.0856, 0.1191, 0.0945, 0.0773, 0.0667, 0.1357,
         0.1118],
        [0.0921, 0.1016, 0.1119, 0.0893, 0.1234, 0.1020, 0.0757, 0.0743, 0.1262,
         0.1035]], grad_fn=<SoftmaxBackward0>)


### Model Parameters

References

* [torch.nn.Module.named_parameters](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.named_parameters)
* [torch.nn.parameter.Parameter](https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html#torch.nn.parameter.Parameter)

In [18]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0038, -0.0354,  0.0260,  ..., -0.0174, -0.0068,  0.0180],
        [-0.0060,  0.0270,  0.0019,  ...,  0.0127, -0.0040,  0.0229]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0243, 0.0008], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0371,  0.0392, -0.0186,  ...,  0.0088, -0.0159, -0.0363],
        [-0.0165,  0.0119, -0.0153,  ...,  0.0045, -0.0027, -0.0379]],
       device='cuda:0', grad_fn=<Slic

## Train Model

References

* [How to train a model](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html)


### Loss Function

References

* [torch.nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
* [torch.nn.LogSoftmax](https://pytorch.org/docs/stable/generated/torch.nn.LogSoftmax.html)
* [torch.nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html)

See more on `Softmax` computation at 
* https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html#torch.nn.Softmax
* http://zh.d2l.ai/chapter_linear-networks/softmax-regression.html#subsec-softmax-operation
* http://zh.d2l.ai/chapter_linear-networks/softmax-regression-concise.html#subsec-softmax-implementation-revisited
* And [here](./pytorch-classification.ipynb)

In [19]:
cross_entropy_loss = nn.CrossEntropyLoss(reduction="none")
input = torch.randn(3, 4)
target = torch.empty(3, dtype=torch.long).random_(4)
cross_entropy_loss_out = cross_entropy_loss(input, target)
print(f"{input = }\n{target = }\n{cross_entropy_loss_out = }\n")

softmax = nn.Softmax(dim=1)
softmax_out = softmax(input)
print(f"{softmax_out = }\n{torch.log(softmax_out) = }\n")

# NOTE: log_softmax(input) is equivalent to torch.log(softmax(input))
log_softmax = nn.LogSoftmax(dim=1)
log_softmax_out = log_softmax(input)
print(f"{log_softmax_out = }\n")

# NOTE: nl_loss(log_softmax(input), target) is equivalent to cross_entropy_loss(input, target)
nl_loss = nn.NLLLoss(reduction="none")
nl_loss_out = nl_loss(log_softmax_out, target)
print(f"{nl_loss_out = }\n")

input = tensor([[-0.6477, -0.0884,  0.1480,  0.8500],
        [-0.9798,  0.6597,  0.5251,  0.6036],
        [ 0.6716,  1.2962, -0.1599,  2.5560]])
target = tensor([1, 3, 1])
cross_entropy_loss_out = tensor([1.6854, 1.1593, 1.6665])

softmax_out = tensor([[0.1060, 0.1854, 0.2348, 0.4738],
        [0.0644, 0.3318, 0.2901, 0.3137],
        [0.1012, 0.1889, 0.0440, 0.6659]])
torch.log(softmax_out) = tensor([[-2.2446, -1.6854, -1.4489, -0.7469],
        [-2.7427, -1.1031, -1.2377, -1.1593],
        [-2.2911, -1.6665, -3.1225, -0.4066]])

log_softmax_out = tensor([[-2.2446, -1.6854, -1.4489, -0.7469],
        [-2.7427, -1.1031, -1.2377, -1.1593],
        [-2.2911, -1.6665, -3.1225, -0.4066]])

nl_loss_out = tensor([1.6854, 1.1593, 1.6665])



### Full Implementation

In [20]:
learning_rate = 1e-3

# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # NOTE: Do ensure all input tensors are on the same device as the model
        X = X.to(device)
        y = y.to(device)

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward() # NOTE: Could/Should it be loss.sum().backward()?
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            # NOTE: Do ensure all input tensors are on the same device as the model
            X = X.to(device)
            y = y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # NOTE: Here pred is not mapped to probablities before argmax. It's an optimazation.
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    

In [21]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300204  [   64/60000]
loss: 2.292131  [ 6464/60000]
loss: 2.275311  [12864/60000]
loss: 2.272105  [19264/60000]
loss: 2.257688  [25664/60000]
loss: 2.226018  [32064/60000]
loss: 2.236367  [38464/60000]
loss: 2.204283  [44864/60000]
loss: 2.214050  [51264/60000]
loss: 2.184824  [57664/60000]
Test Error: 
 Accuracy: 34.5%, Avg loss: 2.174559 

Epoch 2
-------------------------------
loss: 2.181268  [   64/60000]
loss: 2.173115  [ 6464/60000]
loss: 2.123961  [12864/60000]
loss: 2.146177  [19264/60000]
loss: 2.091847  [25664/60000]
loss: 2.033934  [32064/60000]
loss: 2.072547  [38464/60000]
loss: 1.993862  [44864/60000]
loss: 2.013166  [51264/60000]
loss: 1.951093  [57664/60000]
Test Error: 
 Accuracy: 52.8%, Avg loss: 1.937472 

Epoch 3
-------------------------------
loss: 1.961562  [   64/60000]
loss: 1.934868  [ 6464/60000]
loss: 1.827079  [12864/60000]
loss: 1.878328  [19264/60000]
loss: 1.762064  [25664/60000]
loss: 1.706519  [32064/600

## Save and Load Model

In [22]:
model_path = "../model/pytorch-tut"

torch.save(model, model_path)

model = torch.load(model_path)
print(model)

# Validate the loaded model
test_loop(test_dataloader, model, loss_fn)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Test Error: 
 Accuracy: 64.2%, Avg loss: 1.113490 

